In [ ]:
# load packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split




In [ ]:
# Data Cleaning and Imputation for the three main predictors: credit score, delinquency rate, and income. We will merge these with the education data on FIPS code
creditScore = pd.read_csv('Pre_cleaned_data/cty_cred_score_rP_gP_pall.csv')
creditScore['FIPS'] = creditScore['cty'].str[3:]
creditScore = creditScore.drop(columns=['cty']).rename(columns={'Average_credit_score_rP_gP_pall': 'avg_credit_score'})
creditScore['avg_credit_score'] = creditScore['avg_credit_score'].fillna(creditScore['avg_credit_score'].mean())


Delinq = pd.read_csv('Pre_cleaned_data/cty_deliq_rate_rP_gP_p25.csv')
Delinq['FIPS'] = Delinq['cty'].str[3:]
Delinq = Delinq.drop(columns=['cty','Name']).rename(columns={'Debt_Delinquency_rP_gP_p25': 'avg_delinquency_rate'})
Delinq['avg_delinquency_rate'] = Delinq['avg_delinquency_rate'].fillna(Delinq['avg_delinquency_rate'].mean())

Income = pd.read_csv('Pre_cleaned_data/cty_kir_staycz_rP_gP_p25.csv')
Income['FIPS'] = Income['cty'].str[3:]
Income = Income.drop(columns=['cty','Name']).rename(columns={'Individual_Income_Stayed_in_Commuting_Zone_rP_gP_p25': 'avg_income'})
Income['avg_income'] = Income['avg_income'].fillna(Income['avg_income'].mean())

Edu_Data = pd.read_csv('Pre_cleaned_data/EduFund_1996_2003c.csv')
Edu_Data['FIPS'] = Edu_Data['FIPS'].astype(str).str.zfill(5)


left_joined_data = pd.merge(creditScore, Delinq, on='FIPS', how='left')
final_data = pd.merge(left_joined_data, Income, on='FIPS', how='left')
final_data = pd.merge(final_data, Edu_Data, on='FIPS', how='left')

final_data.dropna(inplace=True)
final_data.drop(columns=['SchoolYear','Name','SupportServicesTotal_adj','LocalCharter_adj','StateTransport_adj'], inplace=True)

cols_to_drop = [col for col in final_data.columns if 'PCT' in col]

# Drop them from the DataFrame
final_data.drop(columns=cols_to_drop, inplace=True)



In [ ]:
# fit the model to the training data
X = final_data.drop(columns=["avg_credit_score","avg_delinquency_rate"])
y = final_data["avg_credit_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



selected_columns = X.columns[X.columns.isin(["FederalComp_adj","FederalNutrition_adj","FederalOther_adj","StateFormula_adj","StateOther_adj","PupilSupport_adj"])]
X_train = X_train[selected_columns]
X_test = X_test[selected_columns]
X = X[selected_columns]

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

# check for multicollinearity using VIF
vif_df = pd.DataFrame()
vif_df["feature"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_df)

# fit the OLS model with robust standard errors
ols_model = smf.ols(formula="avg_credit_score ~ FederalNutrition_adj + FederalComp_adj + FederalOther_adj + StateFormula_adj + StateOther_adj + PupilSupport_adj", data=final_data).fit(cov_type='HC3')
print(ols_model.summary())
bp_test = het_breuschpagan(ols_model.resid, ols_model.model.exog)

# check for heteroscedasticity using the Breusch-Pagan test
print("Breusch-Pagan test statistic:", bp_test[0])
print("Breusch-Pagan test p-value:", bp_test[1])






                feature        VIF
0                 const  14.349025
1       FederalComp_adj   1.376581
2  FederalNutrition_adj   1.373790
3      FederalOther_adj   1.121925
4      StateFormula_adj   1.053638
5        StateOther_adj   1.130257
6      PupilSupport_adj   1.058981
                            OLS Regression Results                            
Dep. Variable:       avg_credit_score   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     449.7
Date:                Sun, 22 Feb 2026   Prob (F-statistic):               0.00
Time:                        01:05:39   Log-Likelihood:                -13880.
No. Observations:                3113   AIC:                         2.777e+04
Df Residuals:                    3106   BIC:                         2.782e+04
Df Model:                           6                                         
Covarianc

In [ ]:
OUTCOME_COL = "avg_credit_score"
ID_COL = "FIPS"

PREDICTOR_COLS = [
    "FederalComp_adj",
    "FederalNutrition_adj",
    "FederalOther_adj",
    "StateFormula_adj",
    "StateOther_adj",
    "PupilSupport_adj"
]

ROBUST_SE_TYPE = "HC3"


missing = [c for c in [ID_COL, OUTCOME_COL] + PREDICTOR_COLS if c not in final_data.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")


n_counties = final_data[ID_COL].nunique()
print("Unique counties:", n_counties)


#  Build LONG table 

base = final_data[[ID_COL, OUTCOME_COL] + PREDICTOR_COLS].copy()

county_long = base.melt(
    id_vars=[ID_COL, OUTCOME_COL],
    value_vars=PREDICTOR_COLS,
    var_name="predictor",
    value_name="spend_dollars"
)

# per-$1,000 units (for ROI interpretation)
county_long["spend_k"] = county_long["spend_dollars"].astype(float) / 1000.0

print("Long rows (should be approx unique_counties * 6 if 1 row/county):", county_long.shape[0])


# Fit the multivariate OLS on COMPLETE CASES (unique partial effects)

df_model = base.dropna(subset=[OUTCOME_COL] + PREDICTOR_COLS).copy()

# build X in $1,000 units
X = df_model[PREDICTOR_COLS].astype(float) / 1000.0
X = sm.add_constant(X)
y = df_model[OUTCOME_COL].astype(float)

ols = sm.OLS(y, X).fit()
ols_r = ols.get_robustcov_results(cov_type=ROBUST_SE_TYPE)

betas = pd.Series(ols_r.params, index=ols_r.model.exog_names)

# ROI mapping: predictor -> points per $1,000
roi_map = {p: float(betas.get(p, np.nan)) for p in PREDICTOR_COLS}
county_long["roi_points_per_1000"] = county_long["predictor"].map(roi_map)


# Contributions vs "average county" baseline (continuous)
# Contribution = beta * (x_i - mean(x)) in $1,000 units

means_k = (df_model[PREDICTOR_COLS].astype(float) / 1000.0).mean()  # mean spend_k in the model sample
county_long["mean_spend_k_model"] = county_long["predictor"].map(means_k.to_dict())

county_long["contribution_points_vs_baseline"] = (
    county_long["roi_points_per_1000"] * (county_long["spend_k"] - county_long["mean_spend_k_model"])
)

# Baseline predicted credit for "average county" 
baseline = float(betas.get("const", 0.0) + (betas[PREDICTOR_COLS] * means_k[PREDICTOR_COLS]).sum())
county_long["baseline_pred_credit"] = baseline

def dollars_for_100(b):
    return (1000.0 * (100.0 / b)) if (np.isfinite(b) and b > 0) else np.nan

d100_map = {p: dollars_for_100(roi_map[p]) for p in PREDICTOR_COLS}
county_long["dollars_for_100_credit_points"] = county_long["predictor"].map(d100_map)

# Add model-predicted credit at the county row level 

df_model["_pred_credit"] = ols.predict(X)
county_long = county_long.merge(
    df_model[[ID_COL, "_pred_credit"]],
    on=ID_COL,
    how="left"
).rename(columns={"_pred_credit": "pred_credit"})


# Final checks
print("county_long shape:", county_long.shape)
print("Rows per county (should be 6 if 1 row/county):")
print(county_long.groupby(ID_COL).size().value_counts().head())


Unique counties: 3113
Long rows (should be approx unique_counties * 6 if 1 row/county): 18678
county_long shape: (18678, 11)
Rows per county (should be 6 if 1 row/county):
6    3113
Name: count, dtype: int64


In [ ]:
from auto_sklearn2 import AutoSklearnRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# AutoML with AutoSklearnRegressor
automl = AutoSklearnRegressor(time_limit=100, random_state=42)

automl.fit(X_train, y_train)

y_pred = automl.predict(X_test)
r2 = r2_score(y_test, y_pred)



In [ ]:
# Print the evaluation metrics
print(f"R^2 Score: {r2}")
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"\nBest model: {automl.best_params}")
for model_name, score in automl.get_models_performance().items():
    print(f"{model_name}: {score:.4f}")



# Get the best model and its weight
best_model = automl.best_model
print(best_model)

final_estimator = best_model.steps[-1][1]
print(type(final_estimator))

importances = final_estimator.feature_importances_

# Create a DataFrame to display feature importances
importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)


print(f"Percentage RMSE: {(rmse / y_train.mean()) * 100}")
print(f"Standard Deviation of y_train: {y_train.std()}")

R^2 Score: 0.41908770281376884
RMSE: 2867.6061224539485
MAE: 2060.3134728604273

Best model: {'preprocessor': 'minmax_scaler', 'regressor': 'extra_trees'}
standard_scaler_random_forest: 0.4080
standard_scaler_gradient_boosting: 0.3545
standard_scaler_linear_regression: 0.1970
standard_scaler_ridge: 0.1970
standard_scaler_lasso: 0.1971
standard_scaler_elastic_net: 0.1576
standard_scaler_svr: -0.0140
standard_scaler_knn: 0.3751
standard_scaler_mlp: -24.8130
standard_scaler_decision_tree: -0.2378
standard_scaler_ada_boost: 0.1629
standard_scaler_extra_trees: 0.4120
standard_scaler_bagging: 0.3494
standard_scaler_sgd: 0.1902
standard_scaler_huber: 0.1905
standard_scaler_poisson: 0.1828
standard_scaler_gamma: 0.1196
standard_scaler_tweedie: 0.1204
standard_scaler_ransac: -0.3923
standard_scaler_linear_svr: -26.9500
standard_scaler_kernel_ridge: -32.4218
standard_scaler_pls: 0.1880
minmax_scaler_random_forest: 0.4080
minmax_scaler_gradient_boosting: 0.3550
minmax_scaler_linear_regression: 0.